# <span style="color:#91299A">Основа аспирантской диссертации - проблема наблюдаемости системы</span> 

![](../storage/banners/7.jpg)

### <span style="color:#0ab49a">Согласно статье</span> <span style="color:#A254FC">038 (Shauying R.K.) Observability of Nonlinear Systems</span> 

#### <span style="color:#2c3e50">Начало</span> 

In [148]:
from sympy import *
import numpy as np

Radius_orbit=6750000.
RadiusEarth = 6371000.
mu=398576057600000.06
h_orb = Radius_orbit - RadiusEarth
ω_orb = np.sqrt(mu / (Radius_orbit ** 3))
v_orb = ω_orb * Radius_orbit

t = -131.21 + 0.00299 * h_orb
p = 2.488 * ((t + 273.1) / 216.6) ** -11.388
ρ = p / (0.2869 * (t + 273.1))

print(f"Высота орбиты: {int(h_orb // 1e3)} км\nПериод орбиты: {round((2*np.pi/ω_orb) / 3600, 3)} часов\nПлотность атмосферы: {ρ} кг/м³")

Высота орбиты: 379 км
Период орбиты: 1.533 часов
Плотность атмосферы: 1.1617593224677154e-11 кг/м³


#### <span style="color:#2c3e50">Алгоритм</span>

<span style="color:#2b817d">Примечание:</span>     $km \geq n$

In [180]:
def get_vars(name: str, n: int):
    s = ""
    for i in range(n):
        s += f"{name}_{i} "
    return var(s, real=True)

def get_func(name: str, n: int):
    return [Function(f"{name}_{i}")(t) for i in range(n)]

def get_params(n: int):
    t, ω = var("t ω", real=True)
    x = get_func(f"x", n)
    y = get_func(f"y", n)
    z = get_func(f"z", n)
    vx = get_func(f"v^x", n)
    vy = get_func(f"v^y", n)
    vz = get_func(f"v^z", n)
    wx = get_func(f"w^x", n)
    wy = get_func(f"w^y", n)
    wz = get_func(f"w^z", n)
    q0 = get_func(f"q^0", n)
    qx = get_func(f"q^x", n)
    qy = get_func(f"q^y", n)
    qz = get_func(f"q^z", n)
    return t, ω, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz

def ShauyingObservabilitySufficientCondition(n: int, X_: list, r: list, R: list, my_diff, testprint: bool = False):
    print(f"\033[1mКоличество чипсатов: {n}\033[0m\n")
    
    # Вытягивание измерений в вектор
    Y_ = []
    Y_ += r.copy()
    for i in range(n):
        for j in range(i):
            Y_ += [R[i][j]]
    l = len(Y_)
    k = int(len(X_) // len(Y_))
    print(f"Неизвестные: n = {len(X_)} (на каждый чипсат по {int(len(X_) // n)} параметров)\nИзвестные: l = {l}\n∃ производные порядка k = {len(X_) / len(Y_)}")
    
    # Матрица наблюдаемости системы
    H = Matrix([[Y_[0] for ll in range(l)] for kk in range(k)])
    H_one_line = []
    for kk in range(k):
        for ll in range(l):
            tmp = Y_[ll] if kk==0 else my_diff(H[kk - 1, ll])
            if testprint:
                print(f"_расчёт матрицы H_: k={(kk+1)}/{k}, l={(ll+1)}/{l}")
            H[kk, ll] = tmp
            H_one_line += [tmp]
    H = Matrix(H_one_line)
    print(f"Размерность матрицы H: {shape(H)}")
    
    # Якобиан матрицы наблюдаемости (J[1, 2]: 1 - измерение (H), 2 - состояние (X))
    J = Matrix([[Y_[0] for _ in range(len(X_))] for _ in range(k * l)])
    for k1 in range(k * l):
        for k2 in range(len(X_)):
            J[k1, k2] = H[k1].diff(X_[k2])
    print(f"Размерность матрицы J: {shape(J)}")
    
    # Отречение от символов и знаков, воздадим числам былой языческий огонь
    s_r = lambda: np.random.uniform(-100, 100)
    s_v = lambda: np.random.uniform(-1, 1)
    s_w = lambda: np.random.uniform(-1e-4, 1e-4)
    q = np.array([s_v() for _ in range(4)])
    q /= np.linalg.norm(q)
    rand_params = [(ω, ω_orb)]
    for i in range(n):
        rand_params += [(x[i], s_r()), (y[i], s_r()), (z[i], s_r()), 
                        (vx[i], s_v()), (vy[i], s_v()), (vz[i], s_v()), 
                        (wx[i], s_w()), (wy[i], s_w()), (wz[i], s_w()), 
                        (q0[i], q[0]), (qx[i], q[1]), (qy[i], q[2]), (qz[i], q[3]),
                        (Mx[i], 0), (My[i], 0), (Mz[i], 0)]
    J_numb = J.subs(rand_params)
    
    # Достаточное условие
    print(f"\nСледующие параметры не должны быть нулевыми:\n")
    d = []
    flag = True
    i_min = -1
    for i in range(len(X_)):
        d += [J_numb.minor(i, i) if i == 0 else J_numb.minor(i, i) / d[-1]]
        print(f"Δ_{i} = {d[-1]}")

        # Чек наблюдаемости
        if flag:
            if abs(d[-1]) < 1e-7:
                i_min = i
                flag = False

    # Вывод
    if flag:
        print(f"\n\033[1mВыполнено достаточное условие! Система наблюдаема\033[0m")
    else:
        print(f"\n\033[1mНе выполнено достаточное условие. Нулевой параметр: Δ_{i_min} = {d[i_min]}\033[0m")

    return H, J, J_numb, d

![](../storage/banners/6.gif)

#### <span style="color:#2c3e50">Наблюдаемость системы</span> <span style="color:#00b0b9">при кубсате строго на круговой орбите, без аэродинамики и углового движения,</span> <span style="color:#e60b9d">антенны изотропные</span>

In [181]:
# Количество чипсатов
n = 3

# Инициализация
t, ω, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)
X_ = []
for i in range(n):
    X_ += [x[i], y[i], z[i], vx[i], vy[i], vz[i]]
r = [sqrt(x[i]**2 + y[i]**2 + z[i]**2) for i in range(n)]
R = [[r[i] if i==j else sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2) for i in range(n)] for j in range(n)]

# Уравнения движения
def MyDiff_NoAero_NoAttitude(expr, power: int = 1, vari: any = t):
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = []
    for i in range(n):
        subses += [(Derivative(x[i], t), vx[i])]
        subses += [(Derivative(y[i], t), vy[i])]
        subses += [(Derivative(z[i], t), vz[i])]
        subses += [(Derivative(vx[i], t), -2*ω*vz[i])]  # Нет учёта аэродинамики
        subses += [(Derivative(vy[i], t), -ω**2*y[i])]
        subses += [(Derivative(vz[i], t), 2*ω*vx[i] + 3*ω**2*z[i])]
    return anw.diff(vari).subs(subses).simplify()

H_1, J_1, J_numb_1, d_1 = ShauyingObservabilitySufficientCondition(n=n, X_=X_, r=r, R=R, my_diff=MyDiff_NoAero_NoAttitude)

Количество чипсатов: 3

Неизвестные: n = 18 (на каждый чипсат по 6 параметров)
Известные: l = 6
∃ производные порядка k = 3.0
Размерность матрицы H: (18, 1)
Размерность матрицы J: (18, 18)

Следующие параметры не должны быть нулевыми:

Δ_0 = 8.60231493160760E-20
Δ_1 = -0.593104005282523
Δ_2 = -7.80355246027562E-20
Δ_3 = -8.13792895871826
Δ_4 = 9.19004009018045E-21
Δ_5 = 247.695441534162
Δ_6 = -4.25133099420084E-22
Δ_7 = 522.976477236570
Δ_8 = -1.08341574086129E-22
Δ_9 = 3254.92936270906
Δ_10 = 1.64727410357984E-22
Δ_11 = -89.7623474189247
Δ_12 = 3.83911663805243E-22
Δ_13 = -422.520128334142
Δ_14 = 3.14570405703392E-22
Δ_15 = 145.105532843909
Δ_16 = 4.82793287004358E-21
Δ_17 = 21.3738435836089

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = 8.60231493160760E-20


----

#### <span style="color:#2c3e50">Наблюдаемость системы</span> <span style="color:#00b0b9">при кубсате строго на круговой орбите, с аэродинамикой и угловым движением,</span> <span style="color:#e60b9d">антенны изотропные</span>

In [170]:
# Количество чипсатов
n = 1

# Коэффициент сопротивления
C = 1.17
m = 0.01  # 10 грамм
Jxx, Jyy, Jzz = (0.01, 0.01, 0.007)

# Инициализация
t, ω, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)
X_ = []
for i in range(n):
    X_ += [x[i], y[i], z[i], vx[i], vy[i], vz[i], q0[i], qx[i], qy[i], qz[i], wx[i], wy[i], wz[i]]
r = [sqrt(x[i]**2 + y[i]**2 + z[i]**2) for i in range(n)]
R = [[r[i] if i==j else sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2) for i in range(n)] for j in range(n)]

# Уравнения движения
def MyDiff_NoAero_NoAttitude(expr, power: int = 1, vari: any = t):
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = []
    for i in range(n):
        M = np.zeros(3)
        subses += [(Derivative(x[i], t), vx[i])]
        subses += [(Derivative(y[i], t), vy[i])]
        subses += [(Derivative(z[i], t), vz[i])]
        subses += [(Derivative(vx[i], t), -2*ω*vz[i] - C*ρ/m * (v_orb + vy[i])**2 * (qx[i]*qy[i] - q0[i]*qz[i]))]
        subses += [(Derivative(vy[i], t), -ω**2*y[i])]
        subses += [(Derivative(vz[i], t), 2*ω*vx[i] + 3*ω**2*z[i])]
        subses += [(Derivative(q0[i], t), (-wx[i]*qx[i] - wy[i]*qy[i] - wz[i]*qz[i])/2)]
        subses += [(Derivative(qx[i], t), (wx[i]*q0[i] + wy[i]*qz[i] - wz[i]*qy[i])/2)]
        subses += [(Derivative(qy[i], t), (wy[i]*q0[i] + wz[i]*qx[i] - wx[i]*qz[i])/2)]
        subses += [(Derivative(qz[i], t), (wz[i]*q0[i] + wx[i]*qy[i] - wy[i]*qx[i])/2)]
        subses += [(Derivative(wx[i], t), (Jyy*wy[i]*wz[i] - Jzz*wy[i]*wz[i] + M[0]) / Jxx)]
        subses += [(Derivative(wy[i], t), (-Jxx*wx[i]*wz[i] + Jzz*wx[i]*wz[i] + M[1]) / Jyy)]
        subses += [(Derivative(wz[i], t), (Jxx*wx[i]*wy[i] - Jyy*wx[i]*wy[i] + M[2]) / Jzz)]
    return anw.diff(vari).subs(subses).simplify() # .simplify()

H_2, J_2, J_numb_2, d_2 = ShauyingObservabilitySufficientCondition(n=n, X_=X_, r=r, R=R, my_diff=MyDiff_NoAero_NoAttitude, testprint=True)

Количество чипсатов: 1

Неизвестные: n = 13 (на каждый чипсат по 13 параметров)
Известные: l = 1
∃ производные порядка k = 13.0
_расчёт матрицы H_: k=1/13, l=1/1
_расчёт матрицы H_: k=2/13, l=1/1
_расчёт матрицы H_: k=3/13, l=1/1
_расчёт матрицы H_: k=4/13, l=1/1
Размерность матрицы H: (4, 1)
Размерность матрицы J: (4, 4)

Следующие параметры не должны быть нулевыми:

Δ_0 = 3.49170653028694E-11
Δ_1 = -13.3608967270478
Δ_2 = 1.66090842692955E-8
Δ_3 = 9.54370497871519

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = 3.49170653028694E-11


In [172]:
J_numb_2

Matrix([
[  0.372310719925559,  -0.836265392047091,   -0.402548036751934,                   0],
[0.00105605909988416, 0.00259709475122301, -0.00441854927651588,   0.372310719925559],
[3.49577250104515e-5, 7.40042205199919e-5, -5.26942889459152e-5,   0.001195586156966],
[3.28276741519095e-7, 1.83470871580018e-6, -2.13522513565941e-7, 7.27623285571031e-5]])

### <span style="color:#0ab49a">Согласно книге</span> <span style="color:#A254FC">Афанасьев В.Н.) Математическая теория конструирования систем управления</span> 

In [114]:
A_single = Matrix([[0, 0, 0, 1, 0, 0], 
                   [0, 0, 0, 0, 1, 0],
                   [0, 0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 0, -2*w],
                   [0, -w**2, 0, 0, 0, 0],
                   [0, 0, 3*w**2, 2*w, 0, 0]])
A_ = Matrix([[A_single if i==j else zeros(6, 6) for j in range(n)] for i in range(n)])

In [ ]:
anw = Matrix.hstack(H_, 
                    A_.T @ H_, 
                    A_.T @ A_.T @ H_, 
                    A_.T @ A_.T @ A_.T @ H_, 
                    A_.T @ A_.T @ A_.T @ A_.T @ H_, 
                    A_.T @ A_.T @ A_.T @ A_.T @ A_.T @ H_)
shape(anw)

In [108]:
Radius_orbit = 6800e3
mu = 5.972e24 * 6.67408e-11
w_0 = np.sqrt(mu / Radius_orbit ** 3)

x0rand = [np.random.uniform(-100, 100) for _ in range(n)]
y0rand = [np.random.uniform(-100, 100) for _ in range(n)]
z0rand = [np.random.uniform(-100, 100) for _ in range(n)]
vx0rand = [np.random.uniform(-1, 1) for _ in range(n)]
vy0rand = [np.random.uniform(-1, 1) for _ in range(n)]
vz0rand = [np.random.uniform(-1, 1) for _ in range(n)]

In [109]:
subses = [(w, w_0)]
for i in range(n):
    subses += [(x[0][i], x0rand[i]), (y[0][i], y0rand[i]), (z[0][i], z0rand[i]), (vx[0][i], vx0rand[i]), (vy[0][i], vy0rand[i]), (vz[0][i], vz0rand[i])]
anw_numb = anw.subs(subses)
det(anw_numb)

In [110]:
anw_get = anw_numb[:,0:6]
anw_get

Matrix([
[111.920485834732, 0.371205798574056, 0.00408786854282351,                    0,                    0,                    0],
[115.690689327415, 0.945571542542173, 0.00702484508913071, -7.14198782418648e-5, -6.89198573440637e-7, -1.62727028742046e-8],
[69.3527105132341, -0.33995394940928, 0.00197638505615263, 0.000640287688750991,  2.17169867883479e-6,  1.80664255251497e-8],
[220.143810267416,  1.27720496915373,   0.010815389498481,     112.299618896271,     0.37249172485893,   0.0040985662016115],
[56.3423083929516, 0.543700710847932,  0.0128373453763167,     115.690689327415,    0.945571542542173,  0.00702484508913071],
[168.371828254168, 0.571075913837815, 0.00475079741361419,     68.8569993385628,    -0.34282990933196,   0.0019520313874017]])

In [111]:
det(anw_get)

0.0207281102680228

Линейная система с 1 челиком ненаблюдаема